In [1]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
from tqdm import tqdm

# FULL RUN

In [2]:
accessions= ['at6923','at6929','at7143','at8285','at9104','at9336','at9503','at9578','at9744','at9762','at9806','at9830','at9847','at9852','at9879','at9883','at9900','col0']

pb_augustus_df = pd.DataFrame()
for acc in tqdm(accessions):

    # -------------1-------------
    # prepare sqanti output
    sqanti = pd.read_csv("/ebio/abt6_projects7/dliso/dlis/leon_pipeline/output/v2/collapse_pb/{}_augustus_sqanti/{}-pb-vs-augustus_classification.txt".format(acc,acc), sep="\t")
    sub = sqanti[["isoform","exons","length","structural_category","associated_gene","associated_transcript","subcategory","ORF_seq"]]
    sub["gene"] = sub["isoform"].str.split(".").str[0]+"."+sub["isoform"].str.split(".").str[1]
    # -------------2-------------
    # prepare read support
    reads=pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/output/v2/collapse_pb/{acc}.collapsed.pb.v2.abundance.txt",sep="\t", skiprows=3)
    r_sub = reads[["pbid","fl_assoc"]]
    r_sub.columns = ["isoform", "isoform_reads"]
    r_sub["gene"]=r_sub["isoform"].str.split(".").str[0]+"."+r_sub["isoform"].str.split(".").str[1]
    for i in r_sub.groupby("gene"):
        r_sub.loc[r_sub["gene"]==i[0], "gene_reads"] = i[1]["isoform_reads"].sum()
    r_sub["gene_reads"] = r_sub["gene_reads"].astype("int")

    # -------------3------------
    # add accession ID and merge all information
    merged_all = pd.merge(left=sub, right=r_sub[["isoform","isoform_reads","gene_reads"]], how='left', left_on='isoform', right_on='isoform')
    merged_all['accession'] = acc

    pb_augustus_df = pd.concat([pb_augustus_df, merged_all])

pb_augustus_df = pb_augustus_df[["gene","isoform","accession","associated_gene","associated_transcript","exons","length","isoform_reads","gene_reads","structural_category","subcategory","ORF_seq"]]
pb_augustus_df.columns = ["gene","isoform","accession","associated_gene","associated_transcript","exons","length","isoform_read_support","gene_read_support","structural_category","subcategory","Sqanti_ORF_seq"]

pb_augustus_df.to_pickle("all_runs/pb_augustus_sqanti.pkl")
pb_augustus_df.to_csv("all_runs/pb_augustus_sqanti.tsv", sep="\t")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [1:24:30<00:00, 281.72s/it]


In [8]:
test=pd.read_pickle("pb_augustus_sqanti_full.pkl")

In [ ]:
sns.displot(test, x="exons", hue = "accession", kind="kde",common_norm=False)
plt.savefig("pb_augustus_exons_full.png", bbox_inches='tight')
plt.close()

In [ ]:
sns.displot(test, x="length", hue = "accession", kind="kde",common_norm=False)
plt.savefig("pb_augustus_length_full.png", bbox_inches='tight')
plt.close()